In [ ]:
# ============================================
# 1) 설치
# ============================================
!pip -q install langchain langchain-community langchain-text-splitters pypdf rank-bm25 konlpy

!pip -q install python-mecab-ko


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [ ]:

# ============================================
# 2) PDF 다운로드 (경제금융용어 700선)
# ============================================
import requests, os
import urllib.request

urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")

pdf_path = "2020_경제금융용어 700선_게시.pdf"

if not os.path.exists(pdf_path) or os.path.getsize(pdf_path) == 0:
    r = requests.get(url, stream=True)
    r.raise_for_status()
    with open(pdf_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
print("✅ PDF ready:", pdf_path)

# ============================================
# 3) PDF 로드 & 청크 분할
# ============================================
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"📄 페이지 수: {len(docs)}, 🧩 청크 수: {len(chunks)}")

# ============================================
# 4) 한글 형태소 분석기 (Okt) 적용
# ============================================
from konlpy.tag import Okt
okt = Okt()
mecab = MeCab()

def tokenize_korean(text: str):
    # 형태소 단위로 쪼개고, 불필요한 조사/기호 제거
    return [tok for tok in okt.morphs(text) if tok.strip()]
    #return [w for w in mecab.morphs(text) if w.strip()]


# 테스트
print("형태소 분석 예시:", tokenize_korean("인플레이션은 물가가 오르는 현상을 말합니다."))

# ============================================
# 5) BM25Retriever 구성
# ============================================
from langchain_community.retrievers import BM25Retriever

# BM25Retriever에 사용자 정의 tokenizer를 넘길 수 있음
bm25 = BM25Retriever.from_documents(chunks, preprocess_func=tokenize_korean)
bm25.k = 5  # 상위 5개 문서 검색

# ============================================
# 6) 검색 테스트
# ============================================
def ask(q):
    docs = bm25.get_relevant_documents(q)
    print(f"\n🔎 Q: {q}\n--- 검색 결과 ---")
    for i, d in enumerate(docs, 1):
        page = d.metadata.get("page", "?")
        txt = d.page_content[:150].replace("\n"," ")
        print(f"{i:>2}. p.{page} | {txt}...")

# 테스트 질의
ask("인플레이션의 정의와 원인은 무엇인가요?")
ask("환율 변동이 물가에 미치는 영향은?")
ask("스태그플레이션이란 무엇인가요?")


✅ PDF ready: 2020_경제금융용어 700선_게시.pdf
📄 페이지 수: 371, 🧩 청크 수: 1117
형태소 분석 예시: ['인플레이션', '은', '물가', '가', '오르는', '현상', '을', '말', '합니다', '.']

🔎 Q: 인플레이션의 정의와 원인은 무엇인가요?
--- 검색 결과 ---
 1. p.241 | 경쟁 촉진 등으로 인한 국민후생 증진 등이 기대된다  연관검색어 : 핀테크 인플레이션 물가수준이 지속적으로 상승하는 현상을 인플레이션이라고 한다. 여기서 물가는 개별 상품의 가격을 평균하여 산출한 물가지수를 의미한다. 인플레이션은 물가상승 지속기간  및 상승폭, 제품의...
 2. p.154 | 특성을 지닌 상품이 다수 생산되고 있어 생산비용의 상대적 차이만으로 무역의 발생  원인을 설명하는 데는 한계가 있는 것으로 보인다 .  연관검색어 : 현시비교우위지수 (RCA) 비용인상 인플레이션 재화나 서비스의 생산과 관련하여 투입요소의 비용 상승에 의해 물가가 지속...
 3. p.334 | 개발계획의 입안 및 사업성 검토, 입찰준비, 신용보강 등에 광범위하게 참여하고 다양한  유형의 직간접 금융기법을 결합시키는 투자금융서비스도 제공하게 된다. 유럽 미국 등지 에서는 1960년대부터 많이 사용되어 왔고 우리나라에서는 지난 2000년대 이후 사회간접 자본(S...
 4. p.359 | 343 ABC  있다. 정상회담은 각국 대통령과 총리가 참가하는 가운데 1년에 한번 개최되며 세계의  주요 의제 등을 논의한다. 이 모임은 당초에는 세계 경제의 당면과제 해결방안 모색에  주안점을 두고 있었으나 소련-아프카니스탄 전쟁 발발(1979.12월) 후 1980...
 5. p.117 | 레버리지는 ‘양날의 칼’에 비유되기도 한다. 1997년말 한국에 외환위기가 닥쳤을 때  많은 사업자와 기업이 파산하게 된 원인 중 하나가 높은 부채비율로 인한 레버리지였다.  그러므로 투자를 할 때는 감내할 만한 수준 이내로 부채를 적절하게 사용하는 것이  바람

/tmp/ipython-input-134101655.py:58: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = bm25.get_relevant_documents(q)
